# Solubility Prediction and Model Analysis

In this notebook we're going to be using a dataset of solubility measures to try and train a model to predict a chemical's solubility. The training here is very minimal, and the model being used is very simple. As such, we will not see incredible results, but it will be enough to gain some insight. To gain this insight, once the model is trained, we will check what bonds the most important feature of the model corresponds to.

First, we'll need to make sure we have all of this installed:

In [ ]:
!pip install optuna rdkit sklearn numpy pandas matplotlib joblib tqdm

In [ ]:
# Import necessary libraries
import os
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np
from joblib import dump
from tqdm import tqdm
import optuna
from rdkit.Chem import PeriodicTable
import IPython.display as display

Now that we have everything imported, we'll need to load our dataset as a pandas dataframe.

In [ ]:
# Define the dataset filename and URL
dataset_filename = 'delaney.csv'
dataset_url = "https://raw.githubusercontent.com/dataprofessor/data/master/delaney.csv"

# Function to download dataset
def download_dataset(url, filename):
    print(f"Downloading dataset from {url}...")
    data = pd.read_csv(url)
    data.to_csv(filename, index=False)
    print("Download complete.")

# Check if dataset exists, if not, download it
if not os.path.exists(dataset_filename):
    download_dataset(dataset_url, dataset_filename)

# Load the dataset
print("Loading dataset...")
data = pd.read_csv(dataset_filename)

Next, we'll convert the smiles string column into a molecular fingerprint for each molecule.

In [ ]:
# Define a function to generate molecular fingerprints
def generate_fingerprint(mol, radius=2, nBits=1024):
    return AllChem.GetMorganFingerprintAsBitVect(mol, radius, nBits)

print("Generating molecular fingerprints...")
# Convert SMILES to Mol
data['Mol'] = data['SMILES'].apply(Chem.MolFromSmiles)

# Generate fingerprints
data['Fingerprints'] = data['Mol'].apply(generate_fingerprint)

Now we can trim the dataframe into just what we need, and split the data into training and testing sets.

In [ ]:
# Prepare the dataset
print("Preparing dataset...")
X = np.array(list(data['Fingerprints']))
y = data['measured log(solubility:mol/L)']

# Split the dataset
print("Splitting dataset...")
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

Next, to find the best hyperparameters for our training, we'll use optuna to run some test cases. Optuna is specifically designed to find the best hyperparameters for training our model. That being said, this is not a very exhaustive study we'll be doing; it will only test 20 casea dnt ake the best one. If we were actually trying to get highly accurate results, we would want to do a much bigger study and test many more hyperparameters.

In [ ]:
# Define the objective function for Optuna
def objective(trial):
    # Suggest hyperparameters
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    max_depth = trial.suggest_int('max_depth', 5, 20)

    # Train the model
    model = RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth)
    model.fit(X_train, y_train)

    # Evaluate the model
    y_pred = model.predict(X_val)
    rmse = np.sqrt(mean_squared_error(y_val, y_pred))

    return rmse

# Create an Optuna study and optimize the hyperparameters
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=20)

# Print the best hyperparameters
print(study.best_params)

Awesome! We've got our parameters. Now we can train the model using them. We'll also save our model and the metrics, just to show that this is possible.

In [ ]:
# Train the model with the best hyperparameters
model = RandomForestRegressor(**study.best_params)
model.fit(X_train, y_train)

# Define the number of epochs
n_epochs = 20

# Initialize a DataFrame to store the metrics
metrics = pd.DataFrame(columns=['Epoch', 'Train_RMSE', 'Train_R2', 'Test_RMSE', 'Test_R2'])

print("Starting training...")
for epoch in tqdm(range(n_epochs)):
    # Train the model for one epoch
    model.fit(X_train, y_train)

    # Make predictions
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)

    # Evaluate the model
    rmse_train = np.sqrt(mean_squared_error(y_train, y_pred_train))
    r2_train = r2_score(y_train, y_pred_train)

    rmse_test = np.sqrt(mean_squared_error(y_test, y_pred_test))
    r2_test = r2_score(y_test, y_pred_test)

    # Save the metrics for this epoch
    metrics.loc[epoch] = [epoch, rmse_train, r2_train, rmse_test, r2_test]

# Save the model to a file
print("Saving model...")
dump(model, 'solution.joblib')

# Save the metrics to a CSV file
print("Saving metrics...")
metrics.to_csv('metrics.csv', index=False)

This code makea gif out of a graph made for each point in the training, showing how well our model is performing over time. Don't panic if it's not learning much! In my tests, the subset we're using follows some simple patterns, so the model doesn't actually have much learning to do.

In [ ]:
# Create a figure for the animation
fig, ax = plt.subplots()

# Define the animation update function
def update(i):
    ax.clear()
    ax.plot(metrics['Epoch'][:i+1], metrics['Train_RMSE'][:i+1], label='Train RMSE')
    ax.plot(metrics['Epoch'][:i+1], metrics['Test_RMSE'][:i+1], label='Test RMSE')
    ax.legend()

# Create the animation
print("Creating animation...")
ani = animation.FuncAnimation(fig, update, frames=n_epochs, repeat=False)

# Save the animation to a GIF file
print("Saving animation...")
ani.save('animation.gif', writer='imagemagick')

# Clear the current output
display.clear_output()

# Display the GIF file in the notebook
print("Displaying animation...")
from IPython.display import Image
Image(filename="animation.gif")


Very cool. Not sure why it always displays that blank graph, but it's not a big deal, so let's ignore it. Okay, now that we've created a model, we'll load it in to show how you would do that. Then, we'll analyze the model to get the most important features.

In [ ]:
from joblib import load
from sklearn.inspection import permutation_importance
import matplotlib.pyplot as plt
from rdkit.Chem import rdMolDescriptors

# Load the model from the file
print("Loading model...")
model = load('solution.joblib')

# Perform permutation importance
print("Calculating permutation importance...")
results = permutation_importance(model, X_test, y_test, scoring='neg_mean_squared_error')

# Get importance
importance = results.importances_mean


Now we'll calculate what exactly the most important feature corresponds to in terms of bonds being present in the original molecule. We'll plot the results, and then take a look!

In [ ]:

# Initialize an empty dictionary to store the mapping
bit_to_substructure = {}

# Iterate over all molecules in the dataset
print("Generating feature to substructure dictionary...")
for mol in data['Mol']:
    # Get the info object that contains the mapping from substructures to bits
    info = {}
    fp = rdMolDescriptors.GetMorganFingerprintAsBitVect(mol, 2, 1024, bitInfo=info)

    # Iterate over all bits set in the fingerprint
    for bit, substructures in info.items():
        # Iterate over all substructures that set this bit
        for substructure in substructures:
            # Get the atoms in this substructure
            atoms = [mol.GetAtomWithIdx(idx) for idx in substructure]

            # Generate a string representation of the substructure
            substructure_str = '-'.join(sorted(atom.GetSymbol() for atom in atoms))

            # Add the substructure to the mapping
            if bit not in bit_to_substructure:
                bit_to_substructure[bit] = set()
            bit_to_substructure[bit].add(substructure_str)

# Get the index of the most important feature
most_important_feature_idx = np.argmax(importance)

# Get the bonds that correspond to the most important feature
most_important_substructures = bit_to_substructure.get(most_important_feature_idx, 'No corresponding substructure found')
print(f"The most important feature is feature {most_important_feature_idx}, which has an importance score of {importance[most_important_feature_idx]} and corresponds to these bonds being present in the molecule:\n{most_important_substructures}")

# Plot feature importance
plt.bar([x for x in range(len(importance))], importance)
plt.show()

If everything went correctly, you should see some highly polar bonds that correspond to a higher solubility. 

Thinking point: What could we do to improve this model's performance?